<div style="text-align: right">python==3.12.4</div>
<div style="text-align: right">Last edited by Jun Zhu (zhujun2316@mail.ustc.edu.cn)</div>

## AI for seismic data preprocessing
1. Phase picking
2. Seismic source classification

## phase picking (USTC-Pickers)

In [ ]:
#Step 0： install the packages required by USTC-Pickers
#!pip install seisbench==0.8.0 #uncomment this line if this is your first time to open the notebook

In [ ]:
#Step 1： load the USTC-picker from seisbench
import seisbench.models as sbm
picker = sbm.PhaseNet.from_pretrained('diting')
#print(picker) # uncomment this line to see the structure of the picker

In [ ]:
#Step 2： prepare the data： we'll download a triggered waveform from IRIS （make sure your device has a good connection）
from obspy.clients.fdsn import Client
client = Client('GFZ')
from obspy import UTCDateTime
import matplotlib.pyplot as plt
t = UTCDateTime("2007/01/02 05:48:50")
st = client.get_waveforms(network="CX", station="PB01", location="*", channel="HH?", starttime=t-100, endtime=t+100)
#st.plot();plt.close() # uncomment this line to see the waveform shape

In [ ]:
#Step 3 (optional): visualize the P/S probability reponse of the waveforms predicted by USTC-Picker
response = picker.annotate(st)
#response.plot();plt.close() # uncomment this line to see the response

In [ ]:
#Step 4: pick P or S wave arrivals based on a preset probability threshold
output = picker.classify(st, P_threshold=0.5, S_threshold=0.5)
picks = output.picks
for pick in picks:
    print('\n'*1+'trace ID:',pick.trace_id,
          '\nphase type:',pick.phase,
          '\nprobability:',pick.peak_value,
          '\nUTC time of the arrival:',pick.peak_time)

## Seismic source classification (earthquake vs. quarry blast)

In [ ]:
# Step0: load the classifier
import torch
classifier = torch.load('cnn6.pt',map_location='cpu')

In [ ]:
# Step1: prepare the data, we'll download the waveforms from SCEDC
import os,glob
import numpy as np
def routine_process(st):
    st.detrend('demean').detrend('linear').taper(.05,max_length=5).filter(
        'bandpass',freqmin=1,freqmax=15)

def plot(st,resp,arr,ot):
    _,ax = plt.subplots(4,1,sharex=True)
    ps_color = 'rb'
    ref = ot
    for a in ax[:-1]:
        a.axvline(x=arr-ref,color=ps_color[0],ls='--')
    for i,tr in enumerate(st):
        shift = tr.stats.starttime - ref
        ax[i].plot(tr.times()+shift,tr.data,lw=.8,color='gray',label=tr.stats.channel)
        ax[i].legend()
    ax[3].axhline(y=.3,color='k',ls='--')
    for i,re in enumerate(resp[1:]):
        shift = re.stats.starttime - ref
        ax[3].plot(re.times()+shift,re.data,color=ps_color[i],lw=.8)
    ax[3].set_ylim(0,1);ax[3].set_xlim(st[0].stats.starttime-ref,st[0].stats.endtime-ref)
    ax[3].set_xlabel('Time (s)')
    ax[0].set_title(str(ref))
    plt.subplots_adjust(hspace=0)
    plt.show()
    plt.close()

def cut_and_save(st,arrival,origin_time,eventtype,folder='tmp'):
    if not os.path.exists(folder):os.makedirs(folder)
    data = np.zeros((3,5500))
    tmp = st.copy()
    for i,tr in enumerate(tmp):
        tr = tr.trim(starttime=arrival-2.5,endtime=arrival-2.5+55)
        data[i] += tr.data[:-1]
        trace_id = '%s.%s'%(tr.stats.network,tr.stats.station)
    fname = os.path.join(folder,'%s_%s.npz'%(str(arrival),trace_id))
    np.savez(fname,eventtype=eventtype,data=data,trace_id=trace_id,
             origin_time=origin_time.timestamp,arrival=arrival.timestamp)
    return fname

from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from obspy.geodetics.base import locations2degrees as loc2deg
from obspy.taup import TauPyModel
model = TauPyModel(model='prem')
import seisbench.models as sbm
picker = sbm.PhaseNet.from_pretrained('diting');picker.sampling_rate=50
import matplotlib.pyplot as plt
client = Client('SCEDC')
print(client)
eq,qb = 'earthquake','quarry blast'
temp_dir = 'tmp'
for x in glob.glob(os.path.join(temp_dir,'*')):os.unlink(x)
starttime=UTCDateTime('20221201T19:54:29')
events = client.get_events(starttime=starttime,endtime=starttime+40*60)
print(events)
for event in events:
    origin = event.preferred_origin()
    print(event.event_type,origin.time,origin.longitude,origin.latitude,origin.depth)
    stations = client.get_stations(starttime=origin.time,endtime=origin.time+120,
                                   latitude=origin.latitude,longitude=origin.longitude,
                                   maxradius=1,network='CI')
    print(stations)
    for network in stations.networks:
        for station in network:
            #if station.code!='BOR':continue #comment this line if you want to download more seismograms
            gcarc = loc2deg(station.latitude,station.longitude,
                            origin.latitude,origin.longitude)
            arrivals = model.get_travel_times(
                source_depth_in_km=max(origin.depth/1e3,0),
                distance_in_degree=gcarc,
                phase_list=['P','p'])
            if arrivals:tt=arrivals[0].time
            else:continue
            arrival = origin.time + tt
            print('====try to fetch data====',arrival,network.code,station.code,)
            try:
                st = client.get_waveforms(
                    network.code,station.code,'*','HH*',
                    origin.time-30,origin.time+120)
            except:
                print('No data')
                continue
            if len(st)!=3:continue
            output = picker.classify(st,S_threshold=1);picks = output.picks
            if len(picks)==0:continue
            else:
                print(picks)
            response = picker.annotate(st)
            routine_process(st)
            absdiff = []
            for i,pick in enumerate(picks):absdiff.append(abs(pick.peak_time-arrival))
#            if min(absdiff)<3:continue
            plot(st,response,arrival,origin.time) # uncomment this line to visualize the selected waveforms
            cut_and_save(st,arrival,origin.time,event.event_type)

In [ ]:
# Step2: load the input waveforms and predict earthquake vs. quarry blast probability
import numpy as np
import glob,torch
import torch.nn.functional as F
from obspy import UTCDateTime
import matplotlib.pyplot as plt
from scipy import signal
import pandas as pd
from IPython.display import display
device = 'cpu'
model = torch.load('cnn6.pt',map_location=lambda storage,loc:storage)
model.to(torch.device(device))
flst = glob.glob('tmp/*')

def normalize(data):
    data -= np.mean(data,-1,keepdims=True)
    data = signal.detrend(data,axis=-1)
    data /= np.std(data,axis=-1).max()
    return data
    
def calc_snr(k,win=250,arrival=250):
    x = k.copy()
    noise = x[arrival-win:arrival]
    signal = x[arrival:arrival+win]
    signal -= np.mean(signal)
    noise -= np.mean(noise)
    return 10*np.log10(np.sum(signal*signal)/np.sum(noise*noise))

def plot_3C(d,prob,ground_truth):
    _,ax = plt.subplots(3,1,sharex=True)
    for a,k in zip(ax,d):
        a.axvline(x=250,color='r')
        a.plot(k,color='gray',lw=.8)
    plt.subplots_adjust(hspace=0)
    ax[0].set_title('blast prob.=%.2f (ground truth: %s)'%(prob,ground_truth))
    ax[2].set_xlim(0,5500)
    ax[2].set_xlabel('Sample point')
    plt.show()
    plt.close()
    
def aggregate(df):
    events = {}
    for prob,event,gr in zip(df['blast_probability'],df['origin_time'],df['ground_truth']):
        if event not in events:
            events[event] = {'ground_truth':str(gr),'probs':[prob]}
        else:events[event]['probs'].append(prob)
    evids,probs,gr = [],[],[]
    for k,w in events.items():
        w['prob'] = np.mean(w['probs'])
        evids.append(k)
        probs.append(w['prob'])
        gr.append(w['ground_truth'])
    return pd.DataFrame({'origin_time':evids,'ground_truth':gr,'averaged_blast_probability':probs})

ground_truths,probs,snrs,evids,trace_ids,fnames,arrivals = [],[],[],[],[],[],[]
for x in flst:
    fnames.append(x)
    d = np.load(x)
    data = normalize(d['data'])
    snr = calc_snr(data[-1])
    snrs.append(snr)
    trace_ids.append(d['trace_id'])
    ground_truths.append(d['eventtype'])
    arrivals.append(d['arrival']-d['origin_time'])
    evids.append(str(UTCDateTime(d['origin_time'])))
    data = torch.Tensor(data[None,:]).to(torch.device(device))
    with torch.no_grad():
        prob = F.softmax(model(data),dim=-1)
        probs.append(prob[0][1].item())
        plot_3C(d['data'],probs[-1],d['eventtype'])#uncomment this line to visualize the waveforms
df = pd.DataFrame({
    'fname':fnames,
    'origin_time':evids,
    'trace_id':trace_ids,
    'ground_truth':ground_truths,
    'blast_probability':probs,
    'snr_Z_P_arrival_in_dB':snrs
})
display(df)
display(aggregate(df))

If you find this tool helpful, please cite this paper:
```bibtex
@article{10.1093/gji/ggad463,
    author = {Zhu, Jun and Fang, Lihua and Miao, Fajun and Fan, Liping and Zhang, Ji and Li, Zefeng},
    title = {Deep learning and transfer learning of earthquake and quarry-blast discrimination: applications to southern California and eastern Kentucky},
    journal = {Geophysical Journal International},
    volume = {236},
    number = {2},
    pages = {979-993},
    year = {2023},
    month = {11},
    issn = {0956-540X},
    doi = {10.1093/gji/ggad463},
    url = {https://doi.org/10.1093/gji/ggad463},
    eprint = {https://academic.oup.com/gji/article-pdf/236/2/979/54450467/ggad463.pdf},
}
```